In [5]:
import re
from tqdm import tqdm
from rdkit.Chem import AllChem
from rdkit import Chem, RDLogger
from itertools import chain, permutations
from multiprocessing import Pool, freeze_support
from collections import defaultdict

def mol_list_to_str(mols):
    '''List of RDKit molecules to string separated by ++'''
    inchis = [Chem.MolToSmiles(mol, allHsExplicit=True,allBondsExplicit=True) for mol in mols]
    return ' ++ '.join(inchis)

def mol_list_from_str(inchis):
    '''string separated by ++ to list of RDKit molecules'''
    return [Chem.MolFromSmiles(inchi.strip()) for inchi in inchis.split('++')]

def convert_to_retro(transform):
    '''This function takes a forward synthesis and converts it to a
    retrosynthesis. Only transforms with a single product are kept, since
    retrosyntheses should have a single reactant (and split it up accordingly).'''    

    # Split up original transform
    reactants = transform.split('>>')[0]
    products  = transform.split('>>')[1]

    # Don't force products to be from different molecules (?)
    # -> any reaction template can be intramolecular (might remove later)
    #products = products[1:-1].replace(').(', '.')

    # Don't force the "products" of a retrosynthesis to be two different molecules!
    #reactants = reactants[1:-1].replace(').(', '.')

    return '>>'.join([products, reactants])

def read(line):
    return line.strip().split('\t')


print('Loading data...')
prod_to_reacs = defaultdict(set)
prod_to_noreacs = defaultdict(set)
Tolri=0
Tolpos=0
Tolneg=0

with open('data/templates_expansion1.dat', 'r') as f:
    for l in tqdm(f, desc='products'):
#    with Pool() as p:
#        for rule, prod, reac in tqdm(p.imap(read, f)):
# rule, prod, reac are strings not lists
            rule, prod, reac = l.strip().split('\t')
            prod_to_reacs[prod].add(reac)
#            print(rule)
            retro_canonical = convert_to_retro(rule)
#            print(retro_canonical)
            rxn = AllChem.ReactionFromSmarts(retro_canonical)
            try:
                outcomes = rxn.RunReactants(mol_list_from_str(reac))
                for outcome in outcomes:
                    for product in outcome:
                
                        try:
                            Chem.SanitizeMol(product)
                            product.UpdatePropertyCache()
                            #create product or reactant using molfromsmarts+sanitizemol is sometimes better than molfromsmiles, but still using molfromsmiles as possible as you can
                            product=Chem.MolFromSmiles(Chem.MolToSmiles(product,allHsExplicit=True,allBondsExplicit=True))
                        except Exception as e:
#                            print('warning1: {}'.format(e))
                            #use pass is not good behavior, however i have validation finally
                            continue
                        if not product:
                            continue
                        prodsmi=Chem.MolToSmiles(product,allHsExplicit=True,allBondsExplicit=True)
                        if  prodsmi != prod:
                            prod_to_noreacs[prodsmi].add(reac)
                            continue
#                        if  prodsmi == prod:
#                            prod_to_reacs[prodsmi].add(reac)
                        #tolri doesnt work well due to repeated same prodsmi but different product   
#                        Tolri+=1
                 
            except Exception as e:
                print('error: {}'.format(e))
                print('rxn: {}'.format(reac))

#print('total right Expansion rules examples:',Tolri)            
transforms=[]  
for prod, reacs in prod_to_reacs.items():
    for reac in reacs:
        transforms.append((prod, reac, '1'))
        Tolpos+=1

for prod, noreacs in prod_to_noreacs.items():
    for noreac in noreacs:
        transforms.append((prod, noreac, '0'))             
        Tolneg+=1
with open('data/inscopedata2.dat', 'w') as f:
    f.write('\n'.join(['\t'.join(rxn_prod) for rxn_prod in transforms]))

print('total positive Expansion rules examples:',Tolpos) 
print('total negative Expansion rules examples:',Tolneg) 

Loading data...


products: 293207it [15:37, 153.83it/s]RDKit ERROR: [21:25:19] Explicit valence for atom # 3 C, 5, is greater than permitted
RDKit ERROR: [21:25:19] Explicit valence for atom # 3 C, 5, is greater than permitted
products: 357929it [20:00, 313.57it/s]RDKit ERROR: [21:29:41] Explicit valence for atom # 3 C, 5, is greater than permitted
RDKit ERROR: [21:29:41] Explicit valence for atom # 3 C, 5, is greater than permitted
products: 388316it [22:08, 95.40it/s] RDKit ERROR: [21:31:49] Explicit valence for atom # 3 C, 5, is greater than permitted
RDKit ERROR: [21:31:49] Explicit valence for atom # 3 C, 5, is greater than permitted
products: 408521it [23:41, 243.51it/s]RDKit ERROR: [21:33:23] Explicit valence for atom # 3 C, 5, is greater than permitted
RDKit ERROR: [21:33:23] Explicit valence for atom # 3 C, 5, is greater than permitted
products: 3143138it [3:45:48, 232.00it/s]


total positive Expansion rules examples: 1153125
total negative Expansion rules examples: 381204


In [ ]:
from neg_data2 import *



if __name__ == '__main__':
    print('Loading data...')
    prod_to_reacs = defaultdict(set)
    prod_to_noreacs = defaultdict(set)
    Tolri=0
    Tolpos=0
    Tolneg=0
    expansion_rules = []
    tem_simp = set()
#    seed =0
#    random.seed(seed)
    
    with open('data/expansion_expansion.dat', 'r') as f:
        for i, l in tqdm(enumerate(f), desc='expansion'):
            rule = l.strip()
            expansion_rules.append(rule)

#    with open('data/templates_expansion1.dat', 'r') as f:
#        for l in tqdm(f, desc='products'):
#                tem_simp.add(l.strip())
#    with open('data/templates_expansion3.dat', 'w') as f:
#        f.write('\n'.join(tem_simp))                           

    with open('data/templates_expansion3.dat', 'r') as f:
    #    for l in tqdm(f, desc='products'):
#        seeds=[a for a in range(len(f.readlines()))]
#        combo[0]=f
#        combo[1]=seeds
        with Pool() as p:
#            for reacs,noreacs in tqdm(p.imap(partial(read, rules=expansion_rules), f)):
            for reacs,noreacs in tqdm(p.imap(read, f)):  
                for reac, values in reacs.items():
                    for value in values:
                        prod_to_reacs[reac].add(value)
                
                for reac, values in noreacs.items():
                    for value in values:
                        prod_to_noreacs[reac].add(value)
                    
    transforms=[]  
    for prod, reacs in prod_to_reacs.items():
        for reac in reacs:
            transforms.append((prod, reac, '1'))
            Tolpos+=1

    for prod, noreacs in prod_to_noreacs.items():
        for noreac in noreacs:
            transforms.append((prod, noreac, '0'))             
            Tolneg+=1
    with open('data/inscopedata2.dat', 'w') as f:
        f.write('\n'.join(['\t'.join(rxn_prod) for rxn_prod in transforms]))

    print('total positive Expansion rules examples:',Tolpos) 
    print('total negative Expansion rules examples:',Tolneg)

RDKit WARNING: [19:35:42] Enabling RDKit 2019.09.3 jupyter extensions


Loading data...


expansion: 55608it [00:00, 796525.00it/s]
659it [00:18, 38.23it/s]

In [ ]:
import re
from tqdm import tqdm
from rdkit.Chem import AllChem
from rdkit import Chem, RDLogger
from itertools import chain, permutations
from multiprocessing import Pool, freeze_support
from collections import defaultdict
import random
from functools import partial

def mol_list_to_str(mols):
    '''List of RDKit molecules to string separated by ++'''
    inchis = [Chem.MolToSmiles(mol, allHsExplicit=True,allBondsExplicit=True) for mol in mols]
    return ' ++ '.join(inchis)

def mol_list_from_str(inchis):
    '''string separated by ++ to list of RDKit molecules'''
    return [Chem.MolFromSmiles(inchi.strip()) for inchi in inchis.split('++')]

def convert_to_retro(transform):
    '''This function takes a forward synthesis and converts it to a
    retrosynthesis. Only transforms with a single product are kept, since
    retrosyntheses should have a single reactant (and split it up accordingly).'''    

    # Split up original transform
    reactants = transform.split('>>')[0]
    products  = transform.split('>>')[1]

    # Don't force products to be from different molecules (?)
    # -> any reaction template can be intramolecular (might remove later)
    #products = products[1:-1].replace(').(', '.')

    # Don't force the "products" of a retrosynthesis to be two different molecules!
    #reactants = reactants[1:-1].replace(').(', '.')

    return '>>'.join([products, reactants])

def read(line):
#    line = combo[0]
#    seed = combo[1]
    expansion_rules = []
    with open('data/expansion_expansion.dat', 'r') as f:
        for i, l in tqdm(enumerate(f), desc='expansion'):
            rule = l.strip()
            expansion_rules.append(rule)
    rules = expansion_rules       
    prod_to_reacs = defaultdict(set)
    prod_to_noreacs = defaultdict(set)
# rule, prod, reac are strings not lists
    rule, prod, reac = line.strip().split('\t')
#    prod_to_reacs[prod].add(reac)
    #   print(rule)
    seed = rule+prod+reac
    random.seed(seed)
#    rulesrad=random.sample(rules, 500)
    random.shuffle(rules)
    rulesrad=rules[:500]
    for r in rulesrad:
        if r== rule: continue
        retro_canonical = convert_to_retro(r)
        #   print(retro_canonical)
        rxn = AllChem.ReactionFromSmarts(retro_canonical)
        rcts= mol_list_from_str(reac)
        if len(rcts) != rxn.GetNumReactantTemplates(): continue
        try:
            outcomes = rxn.RunReactants(rcts)
            if not outcomes: continue
            for outcome in outcomes:
                for product in outcome:
                    
                    try:
                        Chem.SanitizeMol(product)
                        product.UpdatePropertyCache()
                        #create product or reactant using molfromsmarts+sanitizemol is sometimes better than molfromsmiles, but still using molfromsmiles as possible as you can
                        product=Chem.MolFromSmiles(Chem.MolToSmiles(product,allHsExplicit=True,allBondsExplicit=True))
                    except Exception as e:
        #                   print('warning1: {}'.format(e))
                        #use pass is not good behavior, however i have validation finally
                        continue
                    if not product:
                        continue
                    prodsmi=Chem.MolToSmiles(product,allHsExplicit=True,allBondsExplicit=True)
                    if  prodsmi != prod:
                        prod_to_noreacs[prodsmi].add(reac)
                        continue
                    if  prodsmi == prod:
                        prod_to_reacs[prodsmi].add(reac)
                    #tolri doesnt work well due to repeated same prodsmi but different product   
        #               Tolri+=1
                 
        except Exception as e:
            print('error: {}'.format(e))
            print('rxn: {}'.format(reac))
    
    return prod_to_reacs,prod_to_noreacs

if __name__ == '__main__':
    print('Loading data...')
    prod_to_reacs = defaultdict(set)
    prod_to_noreacs = defaultdict(set)
    Tolri=0
    Tolpos=0
    Tolneg=0
    expansion_rules = []
    tem_simp = set()
#    seed =0
#    random.seed(seed)
    
    with open('data/expansion_expansion.dat', 'r') as f:
        for i, l in tqdm(enumerate(f), desc='expansion'):
            rule = l.strip()
            expansion_rules.append(rule)

#    with open('data/templates_expansion1.dat', 'r') as f:
#        for l in tqdm(f, desc='products'):
#                tem_simp.add(l.strip())
#    with open('data/templates_expansion3.dat', 'w') as f:
#        f.write('\n'.join(tem_simp))                           

    with open('data/templates_expansion3.dat', 'r') as f:
    #    for l in tqdm(f, desc='products'):
#        seeds=[a for a in range(len(f.readlines()))]
#        combo[0]=f
#        combo[1]=seeds
        with Pool() as p:
#            for reacs,noreacs in tqdm(p.imap(partial(read, rules=expansion_rules), f)):
            for reacs,noreacs in tqdm(p.imap(read, f)):  
                for reac, values in reacs.items():
                    for value in values:
                        prod_to_reacs[reac].add(value)
                
                for reac, values in noreacs.items():
                    for value in values:
                        prod_to_noreacs[reac].add(value)
                    
    transforms=[]  
    for prod, reacs in prod_to_reacs.items():
        for reac in reacs:
            transforms.append((prod, reac, '1'))
            Tolpos+=1

    for prod, noreacs in prod_to_noreacs.items():
        for noreac in noreacs:
            transforms.append((prod, noreac, '0'))             
            Tolneg+=1
    with open('data/inscopedata2.dat', 'w') as f:
        f.write('\n'.join(['\t'.join(rxn_prod) for rxn_prod in transforms]))

    print('total positive Expansion rules examples:',Tolpos) 
    print('total negative Expansion rules examples:',Tolneg)